In [1]:
import sys, os
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from scipy.stats import ttest_ind_from_stats
from scipy.spatial import distance
import random
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
np.random.seed(0)

In [2]:
K = [2, 4, 8, 16, 32]
data1 = pd.read_csv('digits-embedding.csv', header=None)

In [3]:
d2 = [2, 4, 6, 7]
d3 = [6, 7]
data2 = data1[data1[1].isin(d2)]
data3= data1[data1[1].isin(d3)]
print data2

           0  1          2          3
2          2  4  36.899612 -19.903789
5          5  2  -9.380422  35.386284
9          9  4  17.556870 -26.461154
13        13  6 -31.575834 -16.675480
15        15  7  23.260860  21.212704
16        16  2  -9.140321  27.662932
18        18  6 -21.807741 -15.063460
20        20  4  38.557343 -17.274111
25        25  2  -3.656035  18.290690
26        26  4  23.786820 -17.405021
28        28  2  18.990074  16.966523
29        29  7  16.105786  19.822973
32        32  6 -27.600263 -13.146408
36        36  6 -26.056347 -17.909390
38        38  7  29.030563   4.839418
39        39  6 -30.877709 -15.899511
42        42  7  25.787097   7.042009
52        52  7  32.579745  14.981600
53        53  4  17.230017 -16.869916
58        58  4  34.930159 -15.920397
60        60  4  39.140494 -20.009266
61        61  4  14.853700 -24.089718
62        62  6 -22.073010 -18.713111
64        64  4  32.398902 -25.105450
66        66  6 -25.399635 -19.210116
71        71

In [4]:
def kmeans(data, K, N):
    print N
    label = [0 for i in range(len(data))]
    count = 0
    centroid_idx = np.random.randint(0, N, size=K)
    centroid = []
    for i in centroid_idx:
        centroid.append((data.iloc[i][2], data.iloc[i][3]))
    while True:
        update = 0
        x = [0.0 for i in range(K)]
        y = [0.0 for i in range(K)]
        label_count = [0 for i in range(K)]
        idx = 0
        for index, row in data.iterrows():
            # print index, row[2], row[3]
            coor = (row[2], row[3])
            candidate_label = []
            for c in centroid:
                candidate_label.append(distance.euclidean(coor, c))
            # Get the label of the current point
            l = candidate_label.index(min(candidate_label))
            x[l] += row[2]
            y[l] += row[3]
            label_count[l] += 1
            
            # Check if there's an update in each round
            if l != label[idx]:
                label[idx] = l
                update += 1
            
            idx += 1
                
        count += 1
        
        if count == 50 or update == 0:
            break
        
        # Update the centroid if they does not meet the stop criterion
        for i in range(len(centroid)):
            centroid[i] = (1.0 * x[i]/label_count[i], 1.0 * y[i]/label_count[i])
        
        print count, update
    
    return label, centroid

In [5]:
# Initial the centroid
label_1 = {}
label_2 = {}
label_3 = {}
centroid_1 = {}
centroid_2 = {}
centroid_3 = {}

for k in K:
    print "K =", k
    # Dataset 1
    label_1[k], centroid_1[k] = kmeans(data1, k, len(data1))
    # Dataset 2
    label_2[k], centroid_2[k] = kmeans(data2, k, len(data2))
    # Dataset 3
    label_2[k], centroid_3[k] = kmeans(data3, k, len(data3))

K = 2
20000
1 17750
2 3287
3 2163
4 1068
5 579
6 408
7 329
8 209
9 133
10 100
11 61
12 34
13 18
14 14
15 13
16 17
17 17
18 13
19 16
20 14
21 7
22 8
23 6
24 3
25 1
26 1
7938
1 2959
2 162
3 221
4 350
5 260
6 1
4064
1 405
2 1555
3 3
K = 4
20000
1 15541
2 3389
3 2141
4 934
5 326
6 108
7 57
8 29
9 20
10 16
11 11
12 15
13 11
14 7
15 2
16 1
7938
1 7568
2 1444
3 1438
4 1107
5 93
6 6
4064
1 3589
2 201
3 110
4 79
5 82
6 84
7 77
8 73
9 46
10 29
11 18
12 15
13 13
14 9
15 7
16 7
17 11
18 3
19 3
20 2
K = 8
20000
1 19087
2 2731
3 2212
4 2057
5 1829
6 1532
7 1376
8 1069
9 770
10 630
11 500
12 420
13 423
14 434
15 450
16 506
17 538
18 524
19 476
20 408
21 296
22 231
23 154
24 103
25 83
26 64
27 56
28 38
29 32
30 26
31 31
32 34
33 33
34 28
35 19
36 19
37 17
38 18
39 16
40 9
41 13
42 9
43 1
7938
1 6465
2 607
3 367
4 191
5 87
6 36
7 24
8 14
9 7
10 4
11 4
12 1
13 1
4064
1 3496
2 471
3 398
4 251
5 186
6 133
7 85
8 63
9 39
10 31
11 28
12 20
13 14
14 9
15 5
16 5
17 4
18 8
19 5
20 5
21 7
22 7
23 6
24 3
25 3
26